In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, roc_auc_score
import matplotlib.pyplot as plt
from xgboost import XGBClassifier

ModuleNotFoundError: No module named 'xgboost'

In [ ]:
df = pd.read_csv('df_amostragem_negativa.csv')
df

,hora_sinistro,tipo_via,latitude,longitude,tp_veiculo_bicicleta,tp_veiculo_caminhao,tp_veiculo_motocicleta,tp_veiculo_nao_disponivel,tp_veiculo_onibus,tp_veiculo_outros,tp_veiculo_automovel,Chuva,Sinistro,mes,dia_semana
0,5,0,-22.320106,-49.068455,0,0,1,0,0,0,1,0,0,4,0
1,21,0,-22.351964,-49.060317,0,0,0,0,0,0,1,0,0,8,3
2,22,0,-22.348451,-49.040764,0,0,0,0,0,0,0,0,0,1,2
3,16,0,-22.317594,-49.109013,0,0,0,0,0,0,2,0,0,1,2
4,15,0,-22.317187,-49.075325,0,0,0,0,0,0,0,0,1,7,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24562,18,0,-22.318745,-49.114113,0,0,0,0,0,0,0,0,1,5,0
24563,7,0,-22.316813,-49.076026,0,0,0,0,0,0,1,0,0,1,0
24564,11,0,-22.314252,-49.036992,0,0,0,0,0,0,0,0,0,8,1
24565,21,0,-22.317243,-49.073675,0,0,1,1,0,0,1,0,1,9,0


In [ ]:
y = df["Sinistro"]
X = df.drop(columns=["Sinistro"])

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, stratify=y, random_state=42
)

In [ ]:
model = XGBClassifier(
    n_estimators=300,
    max_depth=6,
    learning_rate=0.1,
    colsample_bytree=0.8,
    eval_metric="auc",
    random_state=42,
    n_jobs=-1,
    use_label_encoder=False
)

model.fit(X_train, y_train)

c:\Users\gabri\AppData\Local\Programs\Python\Python312\Lib\site-packages\xgboost\training.py:183: UserWarning: [18:06:51] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)


,objective,'binary:logistic'
,base_score,None
,booster,None
,callbacks,None
,colsample_bylevel,None
,colsample_bynode,None
,colsample_bytree,0.8
,device,None
,early_stopping_rounds,None
,enable_categorical,False
,eval_metric,'auc'


In [ ]:
y_prob = model.predict_proba(X_test)[:,1]
y_pred = (y_prob >= 0.15).astype(int)

print(classification_report(y_test, y_pred, digits=3))
print("ROC-AUC:", roc_auc_score(y_test, y_prob))

              precision    recall  f1-score   support

           0      0.864     0.124     0.216      3276
           1      0.354     0.961     0.518      1638

    accuracy                          0.403      4914
   macro avg      0.609     0.542     0.367      4914
weighted avg      0.694     0.403     0.317      4914

ROC-AUC: 0.5373384670545842
